<a href="https://colab.research.google.com/github/ev1025/DA_Study/blob/main/BOOK_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D%EC%9D%84%EC%9C%84%ED%95%9CSQL%EB%A0%88%EC%8B%9C%ED%94%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Apache Hadoop**
- HDFS(하둡파일시스템) : 분산처리시스템, 거대 데이터를 작게 분할하여 여러개의 디스크에 분산저장하고 동시에 읽어들여서 병목 해결
- MapReduce : HDFS 위의 데이터의 순서를 맞추는 알고리즘
- Apache Hive : HDFS 위의 데이터를 SQL스러운 인터페이스로로 간단하게 처리해주는 시스템
- Hive의 HiveQL(SQL스러운 쿼리)를 자동으로 MapReduce 잡으로 변환하여 간단하게 병렬분산처리

- **Hive의 특징**
    - Hive는 파일 기반 시스템으로 특정 레코드를 변경하거나 제거하는 것이 어렵고 인덱스도 디폴트로 존재하지 않는다.   
    (수정을 위해 전체 파일을 열어야함)
    - 데이터를 HDFS에 축적해두고 동적으로 정의할 수 있다.   
    (한국, 서울, 강서구라는 데이터를 하나의 문자열이나 3컬럼으로 정의할 수 있다.)
    - 데이터 분석을 위한 풍부한 UDF(User-Defined-Function)을 활용하여 SQL로 구현하기 어려운 문자열 처리를 JAVA로 간단하게 할 수 있다.
    - 쿼리를 실행할 때 HiveQL -> JAVA -> 연산노드 라는 과정을 거치므로 간단한 쿼리도 많은 시간이소요 될 수 있다.   
    (적은 양의 데이터에 부적합)


**Amazon Redshift**
- AWS에서 제공하는 분산 병렬 RDB로 노드와 인스턴스를 직접 설정해주어야한다.
- HIVE가 파일기반분산시스템이라면 Redshift는 그냥 RDB   
(레코드를 업데이트, 제거 할 수 있고 트랜잭션도 지원)
- 접속 인터페이스가 PostgreSQL과 호환
- 초기에 작은 규모로 시작하고 이후에 스케일 변경 가능
- 시간당 비용이며 최소 비용이 1년에 천만원
- 데이터를 물리적으로 저장할 때 레코드별 저장(일반 RDB방식)이 아닌 컬럼별 저장을 한다.   
(압축률이 높음)
- 정규화를 하기보다는 하나의 컬럼에 필요한 데이터를 모두 추가하는형태   
(SELECT *와 같은 형태는 성능이 안좋게 나옴)

**BigQuery**
- 구글 클라우드서비스, 노드와 인스턴스를 관리할 필요가 없고 데이터의 양으로 비용이 발생
- 유료버전의 GA와 구글의 다른 클라우드 서비스와 연동가능
- 레거시SQL(빅쿼리의SQL)과 스탠다드SQL(기존SQL)이 있음
- Redshift처럼 컬럼 지향 스토리지이므로 자주 활용하는 데이터만 모아서 별도 테이블로 분할하거나 필요한 컬럼만 select하는 테크닉이 필요

## 데이터

**데이터의 종류**
1. 업무데이터 : 업무에 필요한 데이터(갱신데이터)
    - 트랜잭션 데이터 
        - 구매데이터, 리뷰데이터, 플레이데이터 등 사용자 행동을 기록한 데이터
        - 상품ID, 회원ID, 수량, 가격 등이 포함
        - 이를 기반으로 리포트를 만들지만 회원의 성별, 주소지 등을 곧바로 추출 할 수 없다.
    - 마스터 데이터
        - 카테고리마스터, 상품마스터처럼 정보가 담긴 데이터
        - 트랜잭션 데이터와 결합하여 리포트를 만들어야한다. 

2. 로그데이터 : 업무에 직접적으로 필요하지는 않지만 분석을 위해 추출하는 데이터
    - 출력 시점의 정보를 축적해두는 누적형 데이터

### 업무 데이터

**업무데이터의 특징**
1. 데이터의 정밀도가 높다
    - 데이터 처리 중 문제가 발생하면 트랜잭션과 롤백이라는 기능으로 문제를 제거할 수 있다.   
    (데이터의 정합성이 보증된다.)
    - 정확한 값이 요구되는 매출 관련 리포트 등을 만들 때 업무 데이터를 사용
2. 갱신형 데이터
    - 매일 다양한 데이터 추가, 갱신, 제거가 실행된다.   
    (추출하는 시점에 따라 데이터가 바뀔 수 있다.)
3. 다뤄야 하는 테이블의 수가 많다.
    - 하나의 테이블을 참조해서는 해당 데이터를 파악 할 수 없다.   
    (ER 다이어그램을 파악해야한다.)

**업무 데이터 축적 방법**
1. 모든 데이터 변경하기
    - 날짜기반 누적 데이터가 아니면 전체 데이터를 한꺼번에 최신화(우편번호 마스터, 상품 카테고리 마스터)
    - 빈번하게 변화하거나 날짜에 따라 변화하는 테이블은 한꺼번에 바꾸면 과거 정보를 잃어버리게 되므로 주의
2. 모든 레코드의 스냅샷을 날짜별로 저장
    - 출력 결과가 추출 시점에 따라 달라지면 신뢰성이 낮아진다. 모든 레코드를 날짜별로 누적하면 신뢰성을 보장할 수 있다.(데이터용량 비효율을 초래)
3. 어제와의 변경 사항만 누적
    - 트랜잭션 데이터 중에서 변경/삭제 없이 계축 추가만 일어나는 테이블은 모든 데이터를 한꺼번에 변경해도 되지만 데이터 전송량과 처리시간을 줄이려면 데이터 차이만 누적하면 됨

**업무데이터 사용**

1. 매출액, 사용자 수 처럼 정확한 값이 필요한 경우
    - 업무데이터는 트랜잭션 기능으로 데이터의 정합성이 보증된다.
    - 로그데이터는 전송 방법에 따라 중간 손실이 발생할 수 있으므로, 정확한 값을 요구할 때는 업무데이터를 사용해야한다. 
2. 방문 횟수, 페이지뷰, 사용자 유도 등의 데이터 분석에는 사용할 수 없음
    - 트랜잭션에 필요하지 않은 데이터를 모두 저장하면 서비스처리에 영향을 줄 수 있기 때문에 저장하지 않음
3. 리포트에는 추출시점을 명확히 기재 할 것
    - 서비스를 제공할 때 정합성을 보장하는 형태로 구성되므로 리포트를 만들 때의 정합성은 완벽하게 보장할 수 없다.

P.45 로그데이터부터